In [1]:
import numpy as np
import cv2

# Team Members: Skyler Gauuan, Isabella Rocha, Benjamin Reichelt


# size of images 330 x 450
one_color = cv2.imread("./one_color.jpg")
two_color = cv2.imread("./two_color.jpg")
three_color = cv2.imread("./three_color.jpg")
four_color = cv2.imread("./four_color.jpg")

color_array = [one_color, two_color, three_color, four_color]
template_array = []

In [2]:
def binaryConversion(color_array):
    binary_array = []
    for color in color_array:
        grayImage = cv2.cvtColor(color, cv2.COLOR_BGR2GRAY)
        (thresh, binaryImage) = cv2.threshold(grayImage, 127, 255, cv2.THRESH_BINARY)
        binary_array.append(binaryImage)
    return binary_array

    

def contourCleanup(img):     

    contours, hierarchy = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    contour_output = cv2.cvtColor(np.zeros(np.shape(img), dtype='uint8'), cv2.COLOR_GRAY2BGR)

    if (len(contours) > 0):
        # Find largest contour
        max_id = max(enumerate(contours), key=lambda x : cv2.contourArea(x[1]))[0]
        max_size = cv2.contourArea(contours[max_id])
        boundrec = cv2.boundingRect(contours[max_id])

        cv2.drawContours(contour_output, contours, max_id, 255, cv2.FILLED, 8)
        (thresh, binary_contour) = cv2.threshold(contour_output, 127, 255, cv2.THRESH_BINARY)
        binary_contour = cv2.cvtColor(binary_contour, cv2.COLOR_BGR2GRAY)
        
        element = np.ones((5,5))
        binary_contour = cv2.dilate(binary_contour, element)
        binary_contour = cv2.dilate(binary_contour, element)
        binary_contour = cv2.dilate(binary_contour, element)
    return binary_contour


def my_skin_detect(src):
    dst = np.zeros(np.shape(src)[:-1], dtype=np.uint8)
    
    mask = np.logical_and.reduce((src[:,:,2] > 100, src[:,:,1] > 50, src[:,:,0] > 20, src[:,:,2] > src[:,:,1], src[:,:,2] > src[:,:,0])) # abs(src[:,:,2] - src[:,:,1]) > 15)
    dst[mask] = 255

    return dst



def templateMatching(src, template_array):
    highestCorrelation = 0
    bestMatch = -1
    finger = 1
    for template in template_array:
        w = template.shape[1]
        h = template.shape[0]
        for scale in np.linspace(0.2, 1.0, 20)[::-1]:
            resized = cv2.resize(src, (int(src.shape[1] * scale), int(src.shape[0] * scale)))
            
            if resized.shape[0] < h or resized.shape[1] < w: 
                break
            
            result = cv2.matchTemplate(resized.astype(np.uint8), template.astype(np.uint8), cv2.TM_CCORR_NORMED)
            
            (_, maxVal, _, _) = cv2.minMaxLoc(result)
            
            if maxVal > highestCorrelation:
                highestCorrelation = maxVal
                bestMatch = finger
        finger += 1
    if (highestCorrelation >= .78):
        dst = np.zeros((1080, 1920, 3), dtype=np.uint8)
        if (bestMatch == 4):
            dst[:,:,] = (0,255,255) # finger 4 = yellow
        elif (bestMatch == 3):
            dst[:,:,] = (128,0,128) # finger 3 = purple
        elif (bestMatch == 2):
            dst[:,:,] = (208,224,64) # finger 2 = turquise
        elif (bestMatch == 1): 
            dst[:,:,] = (0,165,255) # finger 1 = orange
        return dst 
    else:
        return np.zeros((1080, 1920), dtype=np.uint8)
        

In [ ]:
def main():
    binary_array = binaryConversion(color_array)
    for binary in binary_array:
        template_array.append(contourCleanup(binary))
    
    cap = cv2.VideoCapture(2) # changed from 0 to 2 to work with external camera

    # if not successful, exit program
    if not cap.isOpened():
        print("Cannot open the video cam")
        sys.exit()


    cv2.namedWindow("matching", cv2.WINDOW_AUTOSIZE)
    cv2.namedWindow("Skin", cv2.WINDOW_AUTOSIZE)
    
    # read a new frame from video
    ret, frame0 = cap.read()
    if not ret:
        print("Cannot read a frame from video stream")
    
    while(1):
        # read a new frame from video
        ret, frame = cap.read()
        # if not successful, break loop
        if not ret:
            print("Cannot read a frame from video stream")
            break

        
        frame_dst = my_skin_detect(frame)
        cv2.imshow("Skin", frame_dst)
        
        matchedFrame = templateMatching(frame_dst, template_array)
        cv2.imshow("matching", matchedFrame)
        frame0 = frame
    

        # wait for 'esc' key press for 30ms. If 'esc' key is pressed, break loop
        if cv2.waitKey(30) == 27:
            print("esc key is pressed by user")
            break

            
main()